In [271]:
import pandas as pd

In [272]:
data = []

for i in ["carne.csv", "poblacion.csv", "produccion.csv", "rendimiento.csv"]:
    df = pd.read_csv(i)
    df = pd.melt( df, id_vars=["región","year"],
                    value_vars=["ave","ovino","porcino","vacuno","caprino","alpaca","llama"],
                    value_name=i.split(".")[0],
                    var_name="specie"
                )
    data.append(df)
    

In [273]:
data[1].shape

(1344, 4)

In [274]:
keys = set()

for i, d in enumerate(data):
    current = set(zip(d["región"], d["year"], d["specie"]))
    print(f"DF {i} has {len(current)} unique keys")
    
    if i == 0:
        keys = current
    else:
        print("Missing in DF i:", keys - current)
        print("Extra in DF i:", current - keys)
        print("===")
# se coloco LimaP en la tabal de poblacion en algunos años , en el 2018 se coloco M.de Dios

DF 0 has 1344 unique keys
DF 1 has 1344 unique keys
Missing in DF i: {('M.de Dios', 2018, 'vacuno'), ('Lima', 2024, 'alpaca'), ('Lima', 2022, 'alpaca'), ('Lima', 2022, 'ovino'), ('Lima', 2021, 'llama'), ('Lima', 2024, 'ovino'), ('M.de Dios', 2018, 'llama'), ('Lima', 2024, 'porcino'), ('Lima', 2022, 'porcino'), ('Lima', 2023, 'vacuno'), ('Lima', 2023, 'llama'), ('Lima', 2021, 'caprino'), ('Lima', 2021, 'ave'), ('M.de Dios', 2018, 'caprino'), ('M.de Dios', 2018, 'ave'), ('Lima', 2023, 'caprino'), ('Lima', 2023, 'ave'), ('Lima', 2022, 'vacuno'), ('Lima', 2024, 'vacuno'), ('Lima', 2021, 'alpaca'), ('Lima', 2021, 'ovino'), ('M.de Dios', 2018, 'alpaca'), ('M.de Dios', 2018, 'ovino'), ('Lima', 2022, 'llama'), ('Lima', 2024, 'llama'), ('Lima', 2021, 'porcino'), ('M.de Dios', 2018, 'porcino'), ('Lima', 2023, 'alpaca'), ('Lima', 2023, 'ovino'), ('Lima', 2022, 'caprino'), ('Lima', 2024, 'caprino'), ('Lima', 2023, 'porcino'), ('Lima', 2022, 'ave'), ('Lima', 2024, 'ave'), ('Lima', 2021, 'vacuno')}


In [275]:
data = []
# en algunas tablas se coloco LimaP y M.de Dios
for i in ["carne.csv", "poblacion.csv", "produccion.csv", "rendimiento.csv"]:
    df = pd.read_csv(i)
    df["región"] =  df["región"].replace(to_replace=["M.de Dios", "LimaP"], value=["Madre de Dios", "Lima"])
    df = pd.melt( df, id_vars=["región","year"],
                    value_vars=["ave","ovino","porcino","vacuno","caprino","alpaca","llama"],
                    value_name=i.split(".")[0],
                    var_name="specie"
                )
    data.append(df)

In [276]:
keys = set()

for i, d in enumerate(data):
    current = set(zip(d["región"], d["year"], d["specie"]))
    print(f"DF {i} has {len(current)} unique keys")
    
    if i == 0:
        keys = current
    else:
        print("Missing in DF i:", keys - current)
        print("Extra in DF i:", current - keys)
        print("===")

DF 0 has 1344 unique keys
DF 1 has 1344 unique keys
Missing in DF i: set()
Extra in DF i: set()
===
DF 2 has 1344 unique keys
Missing in DF i: set()
Extra in DF i: set()
===
DF 3 has 1344 unique keys
Missing in DF i: set()
Extra in DF i: set()
===


In [277]:
for i,d in enumerate(data):
    if i==0:
        df = data[0]
    else:
        df = pd.merge(left=df, right=d, on=["región", "specie", "year"], how="inner")

In [278]:
df.shape

(1344, 7)

In [279]:
df.head()

,región,year,specie,carne,poblacion,produccion,rendimiento
0,Tumbes,2024,ave,262.910574,183726.0,93692.0,2.806108
1,Piura,2024,ave,31027.129806,3987073.0,13984015.0,2.218757
2,Lambayeque,2024,ave,25241.836631,2510903.0,11388007.0,2.216528
3,La Libertad,2024,ave,305189.102963,25756805.0,138011041.0,2.211338
4,Cajamarca,2024,ave,3438.308384,899534.0,1551093.0,2.216700


In [280]:
df.to_csv("data_page1.csv", index=False)

## get prices of species

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import argparse
import numpy as np
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


def load_especies(sheet_name, name_cuadro):
    excel_path = ["./data/anuario_pecuaria_2024.xlsx",
                "./data/2730346-cuadros-en-excel-del-anuario-produccion-ganadera-y-avicola-2023.xlsx",
                "./data/Cuadros en Excel del anuario _PRODUCCIÓN GANADERA Y AVÍCOLA_ 2022.xlsx",
                "./data/Cuadros en Excel del anuario _PRODUCCIÓN GANADERA Y AVÍCOLA_ 2021.xlsx",
                "./data/Cuadros en Excel del anuario _PRODUCCIÓN GANADERA Y AVÍCOLA_ 2020.xlsx",
                "./data/Cuadros en Excel del anuario _PRODUCCIÓN GANADERA Y AVÍCOLA_ 2019.xlsx",
                #"./data/Cuadros en Excel del anuario _PRODUCCIÓN GANADERA Y AVÍCOLA_ 2018.xls",
                ]
    
    years = ["2024", "2023", "2022", "2021", "2020", "2019"]

    # 2016 don't follow the pattern
    # 2017 the species put in diferente rows when it have two words
    # 2018 put leche vacuno instead of leche vaca#

    especies = ["especie/ producto","ave", "ovino", "porcino", 
               "vacuno", "caprino", "alpaca", 
               "llama", "huevo gallina",
               #"leche vaca",
               "fibra alpaca", "fibra llama", "lana ovino"]

    '''
    Load data
    '''

    data = {}

    print("Loading data ...")
    for path, year  in zip(excel_path, years):
        
        try :
            print(f"processing excel ... {year}")
            df = pd.read_excel(path, sheet_name)

            col0 = df.iloc[:, 0].astype(str).str.lower()
            found = set(col0.str.strip()) & set(especies)
            
            # find the species correctly
            if len(set(especies)-found) > 0 :
                print(f"missing values for {year}")
                print(set(especies)-found)

            # each specie 
            df.iloc[:,0] = df.iloc[:,0].ffill()
            col0 = df.iloc[:, 0].astype(str).str.lower()
            selec_rows = col0.str.strip().isin(especies)            
            
            # select the data
            starts = df.loc[selec_rows].index
            df = df.iloc[starts,:].reset_index(drop=True) # Selecting the frame

            # set name columns
            df.columns = df.iloc[0].astype(str).str.lower()
            df = df[1:].reset_index(drop=True) 
            df.dropna(subset=["variable"], inplace=True)
            df = df[~df["variable"].str.strip().str.contains("Población", na=False)]

            # remove space
            df["especie/ producto"] = df["especie/ producto"].astype(str).str.strip()
            df["variable"] = df["variable"].astype(str).str.strip()
            
            # selection the Price
            df["variable"] = df["variable"].str.lower().replace({
                "precio (s/ x kg) 1/":"precio",
                "precio (s/ x kg)":"precio",
                "precio (s/ / kg) 1":"precio",
                "precio (s/ / kg)":"precio"
            })
            df = df[df.variable=="precio"]            
            df.drop(columns=["variable", "anual"], inplace=True)

            # Pivot
            df = df.melt(id_vars="especie/ producto", 
                    value_vars=['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'set', 'oct', 'nov', 'dic'],
                    var_name="month",
                    value_name="price"
                )
            
            # add year 
            df["year"] = year
            data[year] = df
            
        except ValueError as e:

            print(f"❌ Error loading in excel {path}, {sheet_name}")
    
    print("Joining data frames ...")
    data = pd.concat(data, axis=0, ignore_index=True)
            
    return data

In [415]:
data = load_especies("C -6", "")

Loading data ...
processing excel ... 2024
processing excel ... 2023
processing excel ... 2022
processing excel ... 2021
processing excel ... 2020
processing excel ... 2019
Joining data frames ...


In [416]:
data.to_csv("prices_species.csv")

In [391]:
data[data["year"]=="2021"]

,especie/ producto,month,price,year


In [321]:
df = load_especies("C -6", "")

Loading data ...
processing excel ...
processing excel ...
processing excel ...
processing excel ...
processing excel ...
processing excel ...
Joining data frames ...


In [322]:
df

,,especie/ producto,month,price,year


In [283]:
df.Variable.unique()

array(['Saca (Unidades)', 'Producción carne (t)',
       'Rendimiento (Kg/Unidad)', 'Precio (S/ x Kg) 1/',
       'Gallinas en producción 3/', 'Producción huevo (t)',
       'Rendimiento (Kg/gallina)', 'Precio (S/ x Kg) ',
       'Alpacas esquiladas', 'Producción fibra (t)',
       'Rendimiento (Kg/alpaca)', 'Llamas  esquiladas',
       'Rendimiento (Kg/llama)', 'Ovinos   esquilados',
       'Producción lana (t)', 'Rendimiento (Kg/ovino)'], dtype=object)

In [284]:
df.columns.values

array(['Especie/ producto', 'Variable', 'Anual', 'Ene', 'Feb', 'Mar',
       'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic'],
      dtype=object)

In [285]:
df["Variable"].replace({"Precio (S/ x Kg) 1/":"Precio","Precio (S/ x Kg)":"Precio"}, inplace=True)

/tmp/ipykernel_9776/3246430135.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Variable"].replace({"Precio (S/ x Kg) 1/":"Precio","Precio (S/ x Kg)":"Precio"}, inplace=True)


In [286]:
df = df[df.Variable=="Precio"].drop(columns=["Variable", "Anual"])

In [287]:
df.columns

Index(['Especie/ producto', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul',
       'Ago', 'Set', 'Oct', 'Nov', 'Dic'],
      dtype='object', name=0)

In [288]:
df

,Especie/ producto,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Set,Oct,Nov,Dic
4,Ave,4.69,4.66,4.69,4.68,4.71,4.68,4.66,4.62,4.6,4.6,4.61,4.66
9,Ovino,5.57,5.59,5.61,5.64,5.64,5.62,5.61,5.6,5.58,5.57,5.57,5.56
14,Porcino,6.91,6.9,6.91,6.93,6.94,6.93,6.95,6.96,6.96,6.97,6.98,7.03
19,Vacuno,5.65,5.67,5.68,5.69,5.69,5.69,5.69,5.69,5.68,5.69,5.68,5.69
24,Caprino,4.81,4.81,4.81,4.8,4.81,4.8,4.81,4.81,4.82,4.82,4.82,4.84
29,Alpaca,4.91,4.97,4.98,5.02,4.98,4.96,4.95,4.92,4.89,4.89,4.87,4.8
35,Llama,4.27,4.3,4.29,4.33,4.31,4.29,4.26,4.25,4.25,4.22,4.21,4.22


In [289]:
df = df.melt(id_vars="Especie/ producto", 
        value_vars=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic'],
        var_name="Mpnth",
        value_name="Price"
        )

In [290]:
df["Year"] = 2019

In [291]:
df

,Especie/ producto,Mpnth,Price,Year
0,Ave,Ene,4.69,2019
1,Ovino,Ene,5.57,2019
2,Porcino,Ene,6.91,2019
3,Vacuno,Ene,5.65,2019
4,Caprino,Ene,4.81,2019
...,...,...,...,...
79,Porcino,Dic,7.03,2019
80,Vacuno,Dic,5.69,2019
81,Caprino,Dic,4.84,2019
82,Alpaca,Dic,4.8,2019


In [ ]:


            '''
            # columns treatment
            df.dropna(axis=1, inplace=True, how="all")
            df.columns = df.iloc[0,:]
            df.drop(0, inplace=True, axis=0)
            df.columns = df.columns.astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
            df.columns = df.columns.astype(str).str.lower()
            df = df.loc[:, ~df.columns.str.contains("nan|^unnamed", case=False, na=True)]
            df["year"] = year

            df = df[["región","ave","ovino","porcino","vacuno","caprino","alpaca","llama","year"]]
            data[year] = df
            

    print(f"✅ Data loaded, only {len(data)} was loaded")

    '''
    #Join data
    '''

    ref_year = "2024"
    ref_cols = set(data[ref_year].columns)

    print("verifying columns names ...")

    for year, df in data.items():
        
        if year == ref_year:
            continue

        cols = set(df.columns)
        diff = ref_cols.symmetric_difference(cols)

        if len(diff)>0:
            print(f"\n📝 Differences vs {ref_year} for {year}:")
            print(diff)
            #data.pop(year) # no se puede remover mientras se esta iterando
            print(f"Data frame from {year} was removed")
        
    print("✅ Columns names are the same.")

    print("Joining data frames ...")

    data = pd.concat(data, axis=0)

    print("✅ Successul")

    print("Saving data ...")
    data.to_csv( name_cuadro+".csv" ,index=False)

    print("✅ Data saved")

    return print("end...")